In [ ]:
import matplotlib as mp
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile

%matplotlib inline

In [ ]:
def cos2envelope(fs, offset, samples, start_time, rise_time, duration):
    '''
    Generates cosine-squared envelope. Can handle generating fragments.
    '''
    def env_func(t, phi=0):
        return np.sin(2*np.pi*t*1.0/rise_time*0.25+phi)**2

    t = (np.arange(samples, dtype=np.float32) + offset)/fs

    m_null_pre = (t < start_time)
    m_onset = (t >= start_time) & (t < rise_time)

    # If duration is set to infinite, than we only apply an *onset* ramp.
    # This is used, in particular, for the DPOAE stimulus in which we want
    # to ramp on a continuous tone and then play it continuously until we
    # acquire a sufficient number of epochs.
    if duration != np.inf:
        m_offset = (t >= (start_time+duration-rise_time)) & \
            (t < (start_time+duration))
        m_null_post = t >= (duration+start_time)
    else:
        m_offset = np.zeros_like(t, dtype=np.bool)
        m_null_post = np.zeros_like(t, dtype=np.bool)

    t_null_pre = t[m_null_pre]
    t_onset = t[m_onset]
    t_offset = t[m_offset]
    t_ss = t[~(m_null_pre | m_onset | m_offset | m_null_post)]
    t_null_post = t[m_null_post]

    f_null_pre = np.zeros(len(t_null_pre))
    f_lower = env_func(t_onset, 0)
    f_upper = env_func(t_offset-(duration-rise_time), np.pi/2)
    f_middle = np.ones(len(t_ss))

    concat = [f_null_pre, f_lower, f_middle, f_upper]
    return np.concatenate(concat, axis=-1)


def sam_eq_power(depth):
    return (3.0/8.0*depth**2.0-depth+1.0)**0.5


def sam_eq_phase(delay, depth, direction):
    if depth == 0:
        return 0
    z = 2.0/depth*sam_eq_power(depth)-2.0/depth+1
    phi = np.arccos(z)
    return 2.0*np.pi-phi if direction == 1 else phi


def sam_envelope(offset, samples, fs, depth, fm, delay, direction):
    eq_phase = sam_eq_phase(delay, depth, direction)
    eq_power = sam_eq_power(depth)

    delay_n = np.clip(int(delay*fs)-offset, 0, samples)
    sam_n = samples-delay_n

    sam_offset = offset-delay_n
    t = (np.arange(sam_n, dtype=np.float32) + sam_offset)/fs
    sam_envelope = depth/2.0*np.cos(2.0*np.pi*fm*t+eq_phase)+1.0-depth/2.0

    # Ensure that we scale the waveform so that the total power remains equal
    # to that of an unmodulated token.
    sam_envelope *= 1.0/eq_power

    delay_envelope = np.ones(delay_n)
    return np.concatenate((delay_envelope, sam_envelope))

In [ ]:
fs = 44100

start_time = 0
rise_time = 200e-3
duration = 4
depth = 0.5
delay = 400e-3
direction = 1
fm = 5

t_target = np.arange(4*fs)/fs
sam_env = sam_envelope(0, len(t_target), fs, depth, fm, delay, direction)
cos_env = cos2envelope(fs, 0, len(t_target), start_time, rise_time, duration)
target = np.cos(2*np.pi*1.5e3*t_target) * sam_env * cos_env

sam_env = sam_envelope(0, len(t_target), fs, 0, fm, delay, direction)
cos_env = cos2envelope(fs, 0, len(t_target), start_time, rise_time, duration)
nogo = np.cos(2*np.pi*1.5e3*t_target) * sam_env * cos_env

t = np.arange(8*fs)/fs-2
background = np.random.uniform(-1, 1, size=len(t))*0.5

figure, axes = plt.subplots(1, 2, figsize=(5.3, 2.5), sharex=True, sharey=True)
ax_nogo, ax_go = axes

for ax in axes:
    ax.plot(t, background, c='0.75')
    ax.plot([-4, 10], [0, 0], c='k')
    ax.set_xlabel('Time (sec)')
    for sp in ('left', 'right', 'top'):
        ax.spines[sp].set_visible(False)
    ax.set_yticks([])

ax_go.plot(t_target, target, c='k')
ax_nogo.plot(t_target, nogo, c='k')

ax_go.text(5, 0.275, 'masker', color='k', ha='center', va='center')
ax_go.text(2, 0, 'go trial\nAM', color='w', va='center', ha='center')
ax_nogo.text(2, 0, 'no-go trial\nno AM', color='w', va='center', ha='center')
ax_nogo.axis(ymin=-1.5, ymax=2.5, xmin=-1, xmax=6)

figure.savefig('schematic.png', bbox_inches='tight')

In [ ]:
scaled = target / target.max() * 32767
scaled = scaled.astype('i')
wavfile.write('target.wav', fs, scaled)

In [ ]:
import numpy as np
x = np.random.random(size=10)
figure, axes = plt.subplots(2, 2, sharex=True, sharey=True)
axes[0, 0].plot(x, 'ko', label='line 1')
x = np.random.random(size=10)
axes[0, 0].plot(x, 'r-', label='line 2')
for ax in axes.flat:
    ax.text(1, 1.5, 'Axes\n(i.e., subplot)', ha='left', va='center')
figure.suptitle('Figure')
ax.axis(xmin=0, xmax=10, ymin=0, ymax=2)
axes[0, 0].legend()
Here we have defined `ax` as a variable to refer to our first panel (of our 1x1 plot). If we were working with a figure that had multiple panels it would be useful to assign numbered variables to our different subplots to make them easy to keep track of when making modifications (ex: `ax1`, `ax2`, etc.). Here we are only working with one subplot so `ax` is sufficient.

This is a superior approach to `plt.plot` because it allows you to juggle multiple axes in the same figure. So, get in the habit of using this approach. For documentation on making figures this way try searching Matplotlib documentation online. If you're a bit confused about terminology:



